In [1]:
%matplotlib inline
import matplotlib as mlp
import scipy
import numpy as np
import sympy
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from sklearn.preprocessing import scale
from scipy.stats.stats import spearmanr 
from sklearn.linear_model import LinearRegression
from sklearn import cross_validation
from sklearn import metrics
import scipy.sparse as sps
import math
from bs4 import BeautifulSoup
import requests
import re
import fileinput
# import sys


In [2]:


def clean_text(text):
    """ Removes white-spaces before, after, and between characters

    :param text: the string to remove clean
    :return: a "cleaned" string with no more than one white space between
    characters
    """
    return ' '.join(text.split())

df_book = pd.DataFrame(data={'Book Name' : [],'Author' : [], 'Awards List' : [], 'Genres' : [], 'Characters' : []})

# reload(sys)
# sys.setdefaultencoding('utf-8')

#TODO :
# 1. Clean awards to trim ..more , ..less
# 2. Delimit by commas 
# 3. Make a pandas dataframe and put into it the data for the book

    
# with open('book_was_better.txt','r') as book_better:
#     for url_first in book_better:
i = 0
fin = fileinput.input(files=('book_was_better.txt', 'movie_was_better.txt'),mode='r')
for url_first in fin:

# if 1:
#         url_first  = book_better.readline()
    try:
        if (i>940):
            book_page = requests.get(url_first)

            # Strain the items from the book page
            soup = BeautifulSoup(book_page.content,"lxml")
            print i
            #Book Name

            book_name = soup.find_all('h1',attrs={"class":'bookTitle'})
            book_name = book_name[0].text
            book_name = clean_text(book_name)
            print book_name

            #Author
            author = soup.find_all('a',attrs={"class":'authorName'})[0].text

            #Awards
            awards_string = ""
            for row in soup.find_all('div',attrs={"class":'infoBoxRowItem',"itemprop":'awards'}):
                    awards_string = row.text

            #Genres
            genre_list = []
            for row in soup.find_all('a',attrs={"class":'actionLinkLite bookPageGenreLink'}):
                    genre_list.append(row.text)

            #Characters
            character_list = []
            for row in soup.find_all('a', {'href': re.compile(r'/characters')}):
                character_list.append(row.text)

            #Other editions
            other_editions = soup.find_all('a', {'href': re.compile(r'/work/editions/')})[1].text
            m = re.search('.*\((.*)\)', other_editions)
            if m:
                other_editions = int(m.group(1))

            #Getting number of votes for [5 4 3 2 1] star ratings        
            script_text = []
            for row in soup.find_all('script',attrs={'type':'text/javascript+protovis'}):
                script_text.append(str(row))
            if len(script_text) >0:
                m = re.search('.*renderRatingGraph\(\[(.*)\]\).*', script_text[0])
            if m:
                rating_values_str = m.group(1)
            rating_values_list = [ int(x) for x in rating_values_str.split(",") ]

            #Average star rating
            star = soup.find_all('span',attrs={"class":'average', "itemprop":"ratingValue"})[0].text
            print star

            #num_reviews
            num_reviews = soup.find_all('span',attrs={"class":'value-title'})[0].text
            m = re.search('(.*?)\s', num_reviews)
            if m:
                num_reviews = int(m.group(1).replace(',', ''))
                
            print ("\nnum_reviews = ",num_reviews)


            df = pd.DataFrame({'Book Name':book_name, 'Author':author, 'Awards List':awards_string, 
                               'Genres':[genre_list], 'Characters':[character_list], 'Other editions':other_editions,
                              'Star votes':[rating_values_list], 'Stars':star, 'Num_reviews':num_reviews})
            df_book = df_book.append(df,ignore_index=True)


            if (i%20 == 0):
                df_book.index += (i-19)
                df_book.to_csv('book_info.csv', mode='a', header=(True if i<21 else False), encoding="utf-8")
                df_book = pd.DataFrame()
        i +=1
            
        
                
    except Exception as exception:
        
#                 df.to_csv('skipped.csv', mode='a', header=False, ignore_index=True, encoding="utf-8")
            f = open('skipped_files', 'a')
            f.write(url_first + " "+type(exception).__name__+"\n")
            f.close()
            print type(exception).__name__
            i +=1
            


fin.close()
df_book

941
[]
IndexError
942
[<h1 class="bookTitle" id="bookTitle" itemprop="name">\n      Grace\n</h1>]
Grace
3.77
('\nnum_reviews = ', 601)
943
[<h1 class="bookTitle" id="bookTitle" itemprop="name">\n      The Solitude of Prime Numbers\n</h1>]
The Solitude of Prime Numbers
3.58
('\nnum_reviews = ', 31440)
944
[<h1 class="bookTitle" id="bookTitle" itemprop="name">\n      Venuto al mondo\n</h1>]
Venuto al mondo
4.24
('\nnum_reviews = ', 3007)
945
[<h1 class="bookTitle" id="bookTitle" itemprop="name">\n      Wide Sargasso Sea\n</h1>]
Wide Sargasso Sea
3.56
('\nnum_reviews = ', 41049)
946
[<h1 class="bookTitle" id="bookTitle" itemprop="name">\n      Musashi\n      <a class="greyText" href="/series/98040-musashi">\n          (Musashi Complete)\n</a></h1>]
Musashi (Musashi Complete)
4.42
('\nnum_reviews = ', 9771)
947
[<h1 class="bookTitle" id="bookTitle" itemprop="name">\n      Heavy\n</h1>]
Heavy
4.12
('\nnum_reviews = ', 17)
948
[<h1 class="bookTitle" id="bookTitle" itemprop="name">\n      Fea

,Author,Awards List,Book Name,Characters,Genres,Num_reviews,Other editions,Star votes,Stars
0,Maurice Sendak,"\nNational Book Award for Children's Books, Pi...",Outside Over There,[],"[Childrens, Picture Books, Childrens, Fantasy,...",2689,13,"[1116, 726, 519, 211, 117]",3.93
1,Amy Tan,,"Sagwa, the Chinese Siamese Cat",[],"[Childrens, Picture Books, Childrens, Animals,...",872,11,"[307, 294, 226, 39, 6]",3.98
2,Ian Fleming,,On Her Majesty's Secret Service (James Bond (O...,"[James Bond, Ernst Stavro Blofeld, Marc-Ange D...","[Fiction, Thriller, Spy Thriller, Espionage, A...",11707,89,"[3433, 4674, 3012, 496, 92]",3.93
3,Vera Chapman,,The King's Damosel (The Three Damosels #2),[],"[Fantasy, Mythology, Arthurian, Fiction, Roman...",155,6,"[32, 48, 53, 13, 9]",3.52
4,Ian Fleming,,You Only Live Twice (James Bond (Original Seri...,"[James Bond, Ernst Stavro Blofeld, M, Irma Bun...","[Fiction, Thriller, Spy Thriller, Espionage, M...",8691,87,"[1950, 3314, 2795, 546, 86]",3.75
5,James M. Cain,,Double Indemnity,[],"[Mystery, Mystery, Noir, Fiction, Mystery, Cri...",13528,46,"[5109, 5539, 2379, 399, 102]",4.12
6,Martin Baynton,,Jane and the Dragon (Jane and the Dragon),[],"[Childrens, Picture Books, Fantasy, Childrens]",171,7,"[78, 54, 29, 5, 5]",4.14
7,Ian Fleming,,The Man With the Golden Gun (James Bond (Origi...,"[James Bond, M, Mary Goodnight]","[Fiction, Thriller, Spy Thriller, Espionage, A...",7023,84,"[1117, 2381, 2757, 661, 107]",3.53
8,Charles Perrault,,Perrault's Fairy Tales,"[Bluebeard, Cinderella, Sleeping Beauty, Littl...","[Childrens, Classics, Cultural, France, Fantas...",8259,28,"[3703, 2789, 1472, 227, 68]",4.19
9,Ian Fleming,,Octopussy & the Living Daylights (James Bond (...,[James Bond],"[Fiction, Thriller, Spy Thriller, Espionage]",4600,43,"[769, 1515, 1791, 468, 57]",3.54
